## ACKR1 Gene (FY, DARC, Duffy)

For gene details, including GRCh37 coordinates, see 
https://uswest.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000213088;ph=31051;r=1:159203307-159206500

For specific information on the 'Duffy-null allele' rs2814778 see Hodgson et. al at https://royalsocietypublishing.org/doi/full/10.1098/rspb.2014.0930

In [2]:
import os
import numpy as np
import pysam
from vcf import Reader        # https://pypi.org/project/PyVCF/
from Bio import SeqIO
from pyliftover import LiftOver
from cogent3 import make_table
from selectiontest import selectiontest
import gzip, pickle
import pandas as pd


#Include pathname to repository
projdir = "/Users/helmutsimon/repos/NeutralityTest"
if not os.getcwd() == projdir:
    os.chdir(projdir)
import vcf_1KG

path = "/Users/helmutsimon/"
if not os.getcwd() == path:
    os.chdir(path)
    
print('selectiontest version: ', selectiontest.__version__)

selectiontest version:  0.1.8


We examine the hypothesis that there is a signal of selection in populations for which the 'Duffy-null allele' rs2814778 segregates. In this cell we compute Tajima's D and $\rho $ for the ACKR1 gene for all populations. Note that the 1KG data uses GRCh37 coordinates.

In [2]:
chrom = 1
fname = 'Data sets/1KG variants full/integrated_call_samples_v3.20130502.ALL.panel'
panel_all = pd.read_csv(fname, sep=None, engine='python', skipinitialspace=True, index_col=0)
vcf_filename = 'Data sets/1KG variants full/ALL.chr' + str(chrom) \
                + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
vcf_file = Reader(filename=vcf_filename, compressed=True, encoding='utf-8')
pops = list(set(panel_all['pop']))
result_dict = dict()
for pop in pops:
    print('\nPopulation =', pop)
    panel = panel_all[panel_all['pop'] == pop]
    #Use GRCh37 coordinates for ACKR1 gene.
    sfs, n, non_seg_snps = vcf_1KG.get_sfs(vcf_file, panel, 1, 159173097, 159176290)
    tajd = selectiontest.calculate_D(sfs)
    print('Tajimas D    =', tajd)
    rho = selectiontest.test_neutrality(sfs)
    print('rho          =', rho)
    result_dict[pop] = [tajd, rho]
results = pd.DataFrame(result_dict, index=['tajd', 'rlnt'])
results
results.to_csv('Google Drive/Genetics/Bayes SFS/Neutrality test/gene_ACKR1_1chrom.csv')


Population = GWD
Tajimas D    = -1.7979502791351145
rho          = 1.6479156070286454

Population = PJL
Tajimas D    = -1.139366699803869
rho          = -0.1334854632810405

Population = STU
Tajimas D    = -0.31594875347773155
rho          = -1.6319386051790872

Population = GBR
Tajimas D    = -0.8279284349385483
rho          = -0.5919787239233401

Population = PEL
Tajimas D    = -1.1347290062902557
rho          = 0.5374268953091388

Population = KHV
Tajimas D    = -1.411195494797755
rho          = 0.2951064804779069

Population = CLM
Tajimas D    = 0.45241441210415745
rho          = -3.2691818357256466

Population = CHS
Tajimas D    = -1.9454003293896316
rho          = 1.6977176170674055

Population = IBS
Tajimas D    = 0.055490517473711885
rho          = -2.4812182086908656

Population = ACB
Tajimas D    = -2.0225300746419497
rho          = 1.8420656799800401

Population = ESN
Tajimas D    = -1.8254967914375653
rho          = 1.4322565470654007

Population = JPT
Tajimas D    = -1.98

Find populations with selection according to $\rho $. All are east Asian, African or African ancestry (Barbados).

In [3]:
threshold = -1
selection_tab = results.loc[:,results.loc['rlnt'] > threshold]
select_pops = selection_tab.columns
print(len(pops), len(select_pops))
select_pops

26 18


Index(['GWD', 'PJL', 'GBR', 'PEL', 'KHV', 'CHS', 'ACB', 'ESN', 'JPT', 'MSL',
       'CHB', 'CDX', 'ASW', 'MXL', 'LWK', 'BEB', 'YRI', 'ITU'],
      dtype='object')

Find populations in which the 'Duffy-null allele' rs2814778 occurs.

In [4]:
#First find hg19 coords for the variant
lo = LiftOver('hg38', 'hg19')
rs2814778_hg19loc = lo.convert_coordinate('chr1', 159204893)
print(rs2814778_hg19loc)

chrom = 1
vcf_filename = 'Data sets/1KG variants full/ALL.chr' + str(chrom) \
                + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
vcf_file = Reader(filename=vcf_filename, compressed=True, encoding='utf-8')
snps = vcf_file.fetch(str(chrom), rs2814778_hg19loc[0][1] - 5, rs2814778_hg19loc[0][1] + 5)
for snp in snps:
    if snp.ID == 'rs2814778':
        break
print(snp.ID, snp)
probands = list()
for proband in snp.samples:
    gt = proband.gt_alleles
    if int(gt[0]) + int(gt[1]) > 0:
    #print(proband)
        probands.append(proband.sample)
fname = 'Data sets/1KG variants full/integrated_call_samples_v3.20130502.ALL.panel'
panel_all = pd.read_csv(fname, sep=None, engine='python', skipinitialspace=True, index_col=0)
panel_all.head()
seg_pops = list(set(panel_all.loc[probands]['pop']))
print(len(seg_pops))
seg_pops

[('chr1', 159174683, '+', 20849626768)]
rs2814778 Record(CHROM=1, POS=159174683, REF=T, ALT=[C])
13


['GWD',
 'LWK',
 'TSI',
 'IBS',
 'MSL',
 'PEL',
 'PUR',
 'ACB',
 'YRI',
 'ASW',
 'CLM',
 'MXL',
 'ESN']

What is relation between populations undergoing selection and segregating for the 'Duffy-null allele' rs2814778. We find that rs2814778 occurs in all African populations undergoing selection, but in none of the east Asian populations.

In [5]:
print(set(seg_pops) - set(select_pops))
print(set(select_pops) - set(seg_pops))

{'IBS', 'PUR', 'CLM', 'TSI'}
{'CHS', 'PJL', 'JPT', 'BEB', 'GBR', 'KHV', 'CHB', 'CDX', 'ITU'}


Pool Asian and African populations and attempt to narrow down where selection signal occurs, looking at 800kb segments.

In [6]:
def analyse_region(chrom, start_hg19, interval, pops):
    fname = 'Data sets/1KG variants full/integrated_call_samples_v3.20130502.ALL.panel'
    panel_all = pd.read_csv(fname, sep=None, engine='python', skipinitialspace=True, index_col=0)
    vcf_filename = 'Data sets/1KG variants full/ALL.chr' + str(chrom) \
                    + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
    vcf_file = Reader(filename=vcf_filename, compressed=True, encoding='utf-8')
    panel = panel_all[panel_all['pop'].isin(pops)]
    reps = 10000
    tajd_results = list()
    rho_results = list()
    for segment in range(4):
        seg_start = start_hg19 + segment * interval
        seg_end = seg_start + interval
        sfs, n, non_seg_snps = vcf_1KG.get_sfs(vcf_file, panel, 1, seg_start, seg_end)
        tajd = selectiontest.calculate_D(sfs)
        tajd_results.append(tajd)
        rho = selectiontest.test_neutrality(sfs)
        rho_results.append(rho)
    return rho_results, tajd_results
            
chrom = 1
start_hg19 = 159173097     # 159176290
interval = 800
results = pd.DataFrame()
pops = ['YRI', 'LWK', 'GWD', 'MSL', 'ESN', 'ASW', 'ASW', 'ACB']
#pops = ['LWK', 'GWD', 'ESN', 'ACB']
rho_results, tajd_results = analyse_region(chrom, start_hg19, interval, pops)
results['afr_rho'] = rho_results
results['afr_tajd'] = tajd_results
print('\n')
pops = ['JPT', 'BEB', 'CHS', 'KHV', 'CDX', 'CHB']
#pops = ['JPT', 'CHS',  'CDX']
rho_results, tajd_results = analyse_region(chrom, start_hg19, interval, pops)
results['asia_rho'] = rho_results
results['asia_tajd'] = tajd_results
results

,afr_rho,afr_tajd,asia_rho,asia_tajd
0,2.812617,-1.892764,1.785845,-1.691455
1,0.459977,-1.741939,3.166857,-1.886447
2,2.676828,-1.779740,1.034396,-1.364605
3,2.233412,-1.614551,1.301328,-1.550474


Format result as Latex table.

In [7]:
result = results.copy()
result.columns = pd.MultiIndex.from_arrays([['Africa', 'Africa', 'Asia', 'Asia'], \
                                            ['rho', 'tajd', 'rho', 'tajd']])
newix = [str((start_hg19 + interval * i)) for i in range(4)]
result.index = newix
print(result)
result.insert(loc=0, column='Start of 800-bps segment', value=newix)
t = make_table(data_frame=result, title="caption", \
               header = ['a\\b', 'a\\b', 'a\\b', 'a\\b'], digits=2)
t.write("Downloads/duffy.tex", label="tab:duffy", justify="lcccc")

             Africa                Asia          
                rho      tajd       rho      tajd
159173097  2.812617 -1.892764  1.785845 -1.691455
159173897  0.459977 -1.741939  3.166857 -1.886447
159174697  2.676828 -1.779740  1.034396 -1.364605
159175497  2.233412 -1.614551  1.301328 -1.550474


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cogent3/util/table.py:179: UserWarning: provided rows/header will be over ridden by DataFrame
  "provided rows/header will be over ridden by " "DataFrame"


Calculate appropriate thresholds for $\rho$.

In [8]:
t1 = selectiontest.compute_threshold(600, 10)
t2 = selectiontest.compute_threshold(600, 5)
print(t1, t2)

0.22571374117049814 0.9889009982482213


Identify SNPs in east Asian populations.

In [9]:
fname = 'Data sets/1KG variants full/integrated_call_samples_v3.20130502.ALL.panel'
panel_all = pd.read_csv(fname, sep=None, engine='python', skipinitialspace=True, index_col=0)
vcf_filename = 'Data sets/1KG variants full/ALL.chr' + str(chrom) \
                + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
vcf_file = Reader(filename=vcf_filename, compressed=True, encoding='utf-8')
pops = ['CDX', 'CHS', 'JPT']
panel = panel_all[panel_all['pop'].isin(pops)]
snps = vcf_file.fetch('1', 159173897, 159174697)
count = 0
seg_snps = list()
for record in snps:
    if record.is_snp:
        count += 1
    for proband in record.samples:
        if proband.sample in panel.index:
            gt = proband.gt_alleles
            if int(gt[0]) + int(gt[1]) > 0:
                seg_snps.append(record.ID)
            
seg_snps_asia = list(set(seg_snps))
print(count)
seg_snps_asia

30


['rs183139118;rs71782098',
 'rs563564963',
 'rs560829766',
 'rs182721947',
 'rs190558956',
 'rs535675282',
 'rs41264467',
 'rs558011238',
 'rs374508775',
 'rs3027012',
 'rs188881743']

Identify SNPs in African populations.

In [10]:
seg_snps_asia = seg_snps
pops = ['GWD', 'LWK', 'ESN', 'ACB']
panel = panel_all[panel_all['pop'].isin(pops)]
snps = vcf_file.fetch('1', 159173897, 159174697)
count = 0
seg_snps = list()
for record in snps:
    if record.is_snp:
        count += 1
    for proband in record.samples:
        if proband.sample in panel.index:
            gt = proband.gt_alleles
            if int(gt[0]) + int(gt[1]) > 0:
                seg_snps.append(record.ID)
seg_snps_afr = list(set(seg_snps))
print(count)
print(seg_snps_afr)

30
['rs183139118;rs71782098', 'rs563564963', 'rs56000654', 'rs3027013', 'rs551968599', 'rs114349581', 'rs3027011', 'rs2814778', 'rs185706527', 'rs548953752', 'rs190558956', 'rs540882553', 'rs3027012']


In [11]:
print(set(seg_snps_asia) - set(seg_snps_afr))
print(set(seg_snps_afr) - set(seg_snps_asia))

{'rs560829766', 'rs374508775', 'rs535675282', 'rs41264467', 'rs558011238', 'rs182721947', 'rs188881743'}
{'rs56000654', 'rs3027013', 'rs551968599', 'rs114349581', 'rs3027011', 'rs2814778', 'rs185706527', 'rs548953752', 'rs540882553'}


We check that the ancestral allele T for rs2814778 is a high-confidence call.

In [12]:
pos37 = 159174683 - 1
anc_filename = 'Data sets/human_ancestor_GRCh37_e59/human_ancestor_1.fa'
for seq_record in SeqIO.parse(anc_filename, "fasta"):
    print(seq_record.id)
    print(seq_record[pos37 - 5: pos37 +5].seq)

ANCESTOR_for_chromosome:GRCh37:1:1:249250621:1
TCTTATCTTG


Look for signal of selection in GWD population for the region chr1:159173897-159174697 containing the rs2814778 variant. We compute likelihood ratio for two models M_0 being a selective model as in Hamlin (2000) and M_1 a neutral model. We use an approximate sample from M_0, which was generated by roc_simulation.py.

In [3]:
pop = 'GWD'
n = 113
fname = '/Users/helmutsimon/Google Drive/Genetics/Software/msms/lib/data/sfs_non_neutral_duffyGWD8.pklz'
with gzip.open(fname, 'rb') as q0:  
    q0 = pickle.load(q0)
row_sums = q0.sum(axis=1)
variates0 = q0 / row_sums[:, np.newaxis]
q1 = selectiontest.sample_wf_distribution(n , reps=10000)

chrom = 1
fname = 'Data sets/1KG variants full/integrated_call_samples_v3.20130502.ALL.panel'
panel_all = pd.read_csv(fname, sep=None, engine='python', skipinitialspace=True, index_col=0)
vcf_filename = 'Data sets/1KG variants full/ALL.chr' + str(chrom) \
                + '.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
vcf_file = Reader(filename=vcf_filename, compressed=True, encoding='utf-8')
print('\nPopulation               =', pop)
panel = panel_all[panel_all['pop'] == pop]
#Use GRCh37 coordinates for FY gene.
sfs, n, non_seg_snps = vcf_1KG.get_sfs(vcf_file, panel, 1, 159173897, 159174697)
odds_ratio = selectiontest.test_neutrality(sfs, variates0=variates0, variates1=q1)
print('Log odds ratio               =', odds_ratio)


Population               = GWD
Log odds ratio               = -0.5848595077272247


In [4]:
10 ** -.584


0.2606153549998896

In [15]:
10 ** -0.29

0.5128613839913648